# `ADKit`
# Documentation

*****

## Introduction

Derivatives are ubiquitous in many fields such as engineering design optimization, fluid dynamics and machine learning.
There are in general three ways to calculate the derivatives: automatic differentiation, numeric differentiation, and
symbolic differentiation. Automatic Differentiation (AD) brings a family of techniques that can calculate the partial
derivatives of any function at any point efficiently and accurately. Unlike numeric differentiation, AD does not have
the problem of floating point precision errors, since it calculates the derivative of a simple function, and keeps track of
these derivatives, and there is no need of step sizes. Compared to symbolic differentiation, AD is not as memory intense,
and can be much faster in terms of the calculation. Therefore, AD is an important way to calculate derivatives in
practice.

There are two modes in Automatic Differentiation: forward mode and the reverse mode. In forward mode, the chain rule is applied to each basic operation, and both the variable's value and derivative are calculated along the way, leading to a complete derivative trace. In reverse mode, there is a forward pass, where the intermediate variables are computed and their values and partial derivatives with respect to the previous layer stored in the memory, and also a reverse pass (popularly known as backward propagation), where we propagate back the derivatives with the help of the chain rule.

The software that we design calculates the derivatives given the user’s input using the forward mode/reverse mode of automatic differentiation depending on the user's choice,
and provides the user with an easy way to solve their optimization problem using derivatives.


## Background

At the core of Automatic Differentiation is the principle that functions implemented as computer code can be broken down into elementary functions, ranging from arithmetic operations (e.g. addition, subtraction etc.) and other functions (e.g. power, exponential, sin etc.). Hence, any differentiable function can be interpreted as a composition of different functions. 

For example, given a function, $f = sin^2(2x)$, it can be rewritten as:

$$ f = \phi_1(\phi_2(\phi_3(x))) $$ 

where $$ \phi_1(z) = z^2,   \phi_2(y) = sin(y) \text{ and } \phi_3(x) = 2x$$


In the forward mode, the chain rule can then be applied successively to each elementary component function to obtain the derivative of the function. Using the same example above, let $c$ be a real number:
$$ f'(c) =  \phi_3'(\phi_2(\phi_1(c))) \cdot \phi_2'(\phi_1(c)) \cdot \phi_1'(c)$$

Based on the example above, the derivative, $f'(c)$, can be evaluated based on the following function-derivative pairs at each stage of computing the function:

$$(\phi_1(c), \phi_1'(c))$$

$$(\phi_2(\phi_1(c)), (\phi_2'(\phi_1(c)) \cdot \phi_1'(c)))$$

$$(\phi_3(\phi_2(\phi_1(c))), \phi_3'(\phi_2(\phi_1(c)) \cdot \phi_2'(\phi_1(c)) \cdot \phi_1'(c))$$

Effectively, the forward mode computes the Jacobian-vector product, $Jp$. This decomposition can be represented via a computational graph structure of calculations, requiring initial values to be set for $x_1$, and $x'_1$:

$$x_1 \rightarrow^{\phi_3(x)} x_2 \rightarrow^{\phi_2(x)} x_3 \rightarrow^{\phi_1(x)} y $$

where $$ \phi_1(x) = x^2,   \phi_2(x) = sin(x) \text{ and } \phi_3(x) = 2x$$

At each stage of the function, the derivative of the function with respect to its argument is calculated. The exact values of the function and its derivative are used for the following function-derivative pair of values. An example of the computational trace for the equation $f = sin^2(2x)$ would look like this, for $x = \dfrac{\pi}{6}$. 

| Trace    | Elementary Operation &nbsp;&nbsp;&nbsp;| Derivative &nbsp;&nbsp;&nbsp; | $\left(f\left(a\right),  f^{\prime}\left(a\right)\right)$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
| :------: | :----------------------:               | :------------------------------: | :------------------------------: |
| $x_{1}$  | $\dfrac{\pi}{6}$                       | $1$                | $\left(\dfrac{\pi}{6}, 1\right)$ |
| $x_{2}$  | $2x_{1}$                               | $2\dot{x}_{1}$     | $\left(\dfrac{\pi}{3}, 2\right)$ |
| $x_{3}$  | $\sin(x_{2})$               | $\cos\left(x_{2}\right)\dot{x}_{2}$            | $\left(\dfrac{\sqrt{3}}{2}, 1\right)$ |
| $x_{4}$  | $x_{3}^{2}$                            | $2x_{3}\dot{x}_{3}$                   | $\left(\dfrac{3}{4}, \sqrt{3}\right)$ |

By evaluating the derivative at each step of the chain rule, we eventually obtain the value of the derivative $f'(x) = \sqrt{3}$ at $x = \dfrac{\pi}{6}$, as second entry of the final tuple in the table.

While the above illustrates the forward mode of AD (the focus of our package), AD also has a reverse mode. Without using chain rule, it first does a forward pass to store the partial derivatives, before undertaking a reverse pass, which starts from the final function to be differentiated, $y$. After fixing the derivative of the final function, it then computes the derivative of each component function with respect to its parent function recursively (using chain rule) until the derivative of the function with respect to the basic-level argument (e.g. $x_1$) can be calculated. 

In terms of efficiency, the forward mode is more efficient when the number of functions to evaluate is much greater than the number of inputs, whereas the reverse mode, which computes the Jacobian-transpose-vector-product is more efficient when the number of inputs is much greater than the number of functions.

More details on the reverse mode is covered in the **Extension (Reverse Mode)** section further below in the documentation.

## How to Use 

### How to install `ADKit`

`ADKit` has only `numpy`(v. 1.14.3 or higher) as a pre-installation requirement, which we have included in a `requirements.txt` file in this repository. Following that, `ADKit` may be installed through the Python Package Index like so:

Alternatively, the user may install `ADKit` by cloning the github repository (https://github.com/the-differentiators/cs207-FinalProject.git) or downloading as a zipped archive.

### Importing 

Once installed, the `AutoDiff` module can be imported simply through the following code:

In [1]:
from ADKit import AutoDiff

If the user chooses to clone the Github repository, once the package's Github repo has been downloaded, the `AutoDiff.py` module containing the classes can be imported by a Python file, with its working directory configured to the same directory as `AutoDiff.py`:

### How to use `ADKit` (Forward Mode)

For the purposes of this demo, we will also import `numpy`, which is a requirement for the `ADKit` package, as well as the `Ad_Var` class from the `AutoDiff` module in the `ADKit` package.

In [2]:
import numpy as np
from ADKit.AutoDiff import Ad_Var

#### Using `ADKit` to compute derivative of a scalar function of one variable (forward mode)

Below, we have included a basic demo for a scalar function, given a single input. The function used in the demo is  $f = sin^2(2x)$, which was used for illustration in the *Background* section earlier. Our objective is to use the the `Ad_Var` class to compute the value of the derivative for this function automatically, unlike the manual computational trace drawn out earlier. 

First, we create an instance of the `Ad_Var` object, with the value of $x = \dfrac{\pi}{6}$ assigned to the input variable, `val`.

In [3]:
a = np.pi / 6

x = Ad_Var(a)

The user should note that the `ADKit` package assumes that for a single input, the object being initialised will have a derivative value of 1 (stored as a Class attribute `self._ders`).

Next, we create `f`, which represents the full function. The `Ad_Var` object from the previous code can be used with dunder functions and additional functions within `Ad_Var` class to construct the full function being evaluated. 

In [4]:
f = (Ad_Var.sin(2*x))**2

As the functions are applied to the original `Ad_Var` object `x`, the `_val` and `_ders` attributes of the object are being updated with new values. The object `f`, representing the full function, will have its `_val` and `_ders` attributes containing the actual function and derivative values respectively.

To note: the user also has the ability to manually set function and derivative values outside of instance initialization using the setter methods provided (`set_val` and `set_ders`). In this way, the user has the option to reuse the same objects after resetting the value and derivative(s).

The associated function value and derivative(s) of any `Ad_Var` instance may be retrieved through the `get_val` and `get_ders` functions as shown below:

In [5]:
print(f.get_val(), f.get_ders())

0.7499999999999999 1.7320508075688776


Also, the function value and derivative can be printed by directly printing the `Ad_Var` object associated with the function `f`. 

In [6]:
print(f)

Value = 0.7499999999999999
Derivative = 1.7320508075688776


#### Using `ADKit` to compute the gradient of a scalar multivariate function (forward mode)

If the user wants to calculate the value and the gradient vector of a scalar multivariate function, then each variable must be first instantiated as an `Ad_Var` object, with inputs `val`, the scalar value of that variable, and `ders`, a `numpy` array representing the seed vector which indicates a direction along which the directional derivative of a function will be calculated. An example is shown below:

In [7]:
x = Ad_Var(1, np.array([1, 0, 0]))
y = Ad_Var(2, np.array([0, 1, 0]))
z = Ad_Var(3, np.array([0, 0, 1]))

Then, the user can define the function which consists of the instantiated `Ad_Var` variables. For example, below , we are calculating the value and the gradient of the function $f = sin^2(2x) + z^y$:

In [8]:
f = (Ad_Var.sin(2*x))**2 + z**y
print(f)

Value = 9.826821810431806
Gradient = [-1.51360499  9.8875106   6.        ]


As we can see above, the gradient of the function `f` is a 3-dimensional vector, since `f` is a function of 3 variables. The first dimension of the gradient vector is the directional derivative of `f` along the seed vector $[1, 0, 0]$. Since, `x` was instantiated with this seed vector, the first dimension of the gradient vector corresponds to the partial derivative $\frac{\partial f}{\partial x}$ evaluated at $x=1, y=2, z=3$. Similarly, $y$ was instantiated with the seed vector $[0, 1, 0]$. In this way, the user has indicated that the second element of the gradient of `f` corresponds to $\frac{\partial f}{\partial y}$ evaluated at at $x=1, y=2, z=3$. Similarly, $z$ was instantiated with the seed vector $[0, 0, 1]$, hence the third dimension of the gradient vector corresponds to $\frac{\partial f}{\partial z}$ evaluated at $x=1, y=2, z=3$.

In summary, each variable should be instantiated with a seed vector with dimensions equal to the dimensions of the gradient vector of the target function. For each variable, the values of the seed vector should be 0 except for one value which should be the derivative of that variable such as 1. The index of the element in the seed vector which has nonzero value indicates the index of the gradient vector which stores the partial derivative value of the target function with respect to this specific variable.  For example, if a variable is initiated with a seed vector of $[1,0,0]$, then this should be interpreted as the first variable among the three variables, and its derivative is set to be 1. 

#### Using `ADKit` to compute derivative of a vector-valued multivariate function (forward mode)

The user can also use `ADKit` to calculate the value and the jacobian matrix of a vector-valued function. Again the variables must be instantiated in the same way as discussed above. Then, a vector-valued function can be defined as a numpy array of functions composed of instantiated `Ad_Var` variables. An example is shown below for the vector valued function $f = \begin{bmatrix}
sin^2(2x) + z^y \\
e^x + z
\end{bmatrix}$ for $x = 1, y = 2, z = 3$: 

In [9]:
x = Ad_Var(1, np.array([1, 0, 0]))
y = Ad_Var(2, np.array([0, 1, 0]))
z = Ad_Var(3, np.array([0, 0, 1]))

f = np.array([(Ad_Var.sin(2*x))**2 + z**y, Ad_Var.exp(x) + z])

Then, the user can call `get_jacobian` to get the jacobian matrix of `f` evaluated at $x = 1, y = 2, z = 3$. The first argument of this method is the vector-valued function $f$ defined as a numpy array. The second argument is the dimension of the vector of the functions (in this example the vector-valued function has 2 dimensions). The third argument is the number of variables composing the vector-valued function (in this example vector-valued function is composed of 3 variables, $x,y$ and $z$).

In [10]:
Ad_Var.get_jacobian(f, 2, 3)

array([[-1.51360499,  9.8875106 ,  6.        ],
       [ 2.71828183,  0.        ,  1.        ]])

Also, the user can call `get_values` by passing `f`, to calculate the value of the vector-valued function for the given values of the variables.

In [11]:
Ad_Var.get_values(f)

array([9.82682181, 5.71828183])

Alternatively, the vector valued function can also be defined as a numpy array of other already instantiated functions, as shown below:

In [12]:
g = (Ad_Var.sin(2*x))**2 + z**y
h = Ad_Var.exp(x) + z
f = np.array([g, h])

In [13]:
Ad_Var.get_jacobian(f, 2, 3)

array([[-1.51360499,  9.8875106 ,  6.        ],
       [ 2.71828183,  0.        ,  1.        ]])

In [14]:
Ad_Var.get_values(f)

array([9.82682181, 5.71828183])

#### Using `ADKit` to compute the derivatives of any type of function on a grid of points (forward mode)

In the above examples, the derivative/gradient/jacobian of a function is evaluated at a single point which is defined by the value with which each variable is instantiated. The `AutoDiff` module, however, can be used to evaluate the derivative/gradient/jacobian of a function on a grid of points defined by the user. The first step to do this is again to instantiate the variables with any value (please note that the default value of an `Ad_Var` variable is 1 so the value argument can be skipped).

In [15]:
x = Ad_Var(ders = np.array([1, 0, 0]))
y = Ad_Var(ders = np.array([0, 1, 0]))
z = Ad_Var(ders = np.array([0, 0, 1]))

Then, the user needs to define the function as a string using the same standard syntax used in any of the examples above. For example, if function is $f = sin^2(2x) + z^y$:

In [16]:
f_string = "(Ad_Var.sin(2*x))**2 + z**y"

Then, the user can call `grid_eval` to calculate the gradient and the value of the given function on a grid of points. The first argument passed is the function string. The second argument is a list of strings where each string represents one of the variables used in the function string. The third argument is the list of the already instantiated `Ad_Var` objects which are referenced in the function string. The last argument is a list of lists defining the grid of all possible points that the user wants to calculate the gradient and the value of the function for. For example, below the function and its gradient are evaluated for all possible combinations of $(x, y, z)$ where $x \in \{1, 2\}, y \in \{2, 3\}, z=4$. The function returns a dictionary where each key is one of the points of the grid and the value is a tuple. The first element of the tuple is the value of the function at this point and the second element of the tuple is the gradient of the function evaluated at this point.

In [17]:
Ad_Var.grid_eval(f_string, ['x', 'y', 'z'], [x, y, z], [[1, 2], [2,3], [4]])

{(1, 2, 4): (16.826821810431806,
  array([-1.51360499, 22.18070978,  8.        ])),
 (1, 3, 4): (64.82682181043181,
  array([-1.51360499, 88.72283911, 48.        ])),
 (2, 2, 4): (16.57275001690431,
  array([ 1.97871649, 22.18070978,  8.        ])),
 (2, 3, 4): (64.57275001690431,
  array([ 1.97871649, 88.72283911, 48.        ]))}

The function `grid_eval` can also be used to evaluate the jacobian for vector valued functions at different points. In this case, the string representation of the vector-valued function must be written as a list of functions referencing the already instantiated `Ad_Var` variables. Please note that in this case the string representation corresponds to a list of functions and not a numpy array of functions. For example, if the user wants to evaluate the jacobian of the vector-valued function $f = \begin{bmatrix}
sin^2(2x) + z^y \\
e^x + z
\end{bmatrix}$ at different points, the function string should be defined as follows:

In [18]:
f_string = "[(Ad_Var.sin(2*x))**2 + z**y, Ad_Var.exp(x) + z]"

Then, by calling `grid_eval` on this function string, a dictionary is returned where each key is one of the points of the grid and the value is a tuple. The first element of the tuple is the value of the function at this point and the second element of the tuple is the jacobian of the vector-valued function evaluated at this point.

In [19]:
Ad_Var.grid_eval(f_string, ['x', 'y', 'z'], [x, y, z], [[1, 2], [2,3], [4]])

{(1, 2, 4): (array([16.82682181,  6.71828183]),
  array([[-1.51360499, 22.18070978,  8.        ],
         [ 2.71828183,  0.        ,  1.        ]])),
 (1, 3, 4): (array([64.82682181,  6.71828183]),
  array([[-1.51360499, 88.72283911, 48.        ],
         [ 2.71828183,  0.        ,  1.        ]])),
 (2, 2, 4): (array([16.57275002, 11.3890561 ]),
  array([[ 1.97871649, 22.18070978,  8.        ],
         [ 7.3890561 ,  0.        ,  1.        ]])),
 (2, 3, 4): (array([64.57275002, 11.3890561 ]),
  array([[ 1.97871649, 88.72283911, 48.        ],
         [ 7.3890561 ,  0.        ,  1.        ]]))}

### How to use `ADKit` (Reverse Mode)



As part of the extension to the minimum requirements, we have implemented the reverse mode of Automatic Differentiation. Using a separate class `rAd_Var`, the user is able to implement the reverse mode using the `ADKit` package. The value of using the reverse mode over the forward mode is the increase in efficiency when the number of inputs is much greater than the number of functions.

The user should note that usage of the `rAd_Var` class differs from that of the `Ad_Var` class in the following ways:
* The initialization of a `rAd_Var` instance does not allow for the input of a derivative value. The implementation necessitates that the derivative of the instance is initialized as None. There is the option for the user to manually set the derivative using the `set_ders` function, if they so wish to.


* The derivatives of the `rAd_Var` object obtained using the `get_ders` method will be returned as a numpy array of partial derivative(s) of the input variables, unlike the forward mode, where the final derivative at the given value of the input variables is calculated using the chain rule.


* To obtain a Jacobian matrix for vector functions with multiple real scalar inputs, e.g. $f = \begin{bmatrix}
xy \\
y \\
ln(x^y)\\
\end{bmatrix}$, the user will need to define the functions first before passing them (as Python functions) as arguments for the `get_jacobian` method, together with the variable names and the given values for the inputs, as demonstrated below. 

*Note: In defining the functions, the user should only include variables used in the function as arguments. Adding additional variables would lead to errors in the calculation.*

This difference in the implementation of the Jacobian matrix is because `rAd_Var` objects can only be defined in the context of one function. Hence, feeding all the functions into the `get_jacobian` method allows for the Jacobian to obtain the respective partial derivatives for each function separately, before combining them and returning them in a single Jacobian matrix.

#### Using `ADKit` to compute derivative of a scalar function of one variable (reverse mode)

As a demo, we will again find the value and the derivative of $f = sin^2(2x)$ at $x = \dfrac{\pi}{6}$, similar to the demo for `Ad_Var` above.

In [20]:
from ADKit.AutoDiff import rAd_Var

In [21]:
a = np.pi / 6

x = rAd_Var(a)

f = (rAd_Var.sin(2*x))**2

print(f)

Value = 0.7499999999999999
Partial Derivative(s) = [1.73205081]


#### Using `rAd_Var` to compute derivative, values of a vector-valued multivariate function (reverse mode)

We will use a similar function as shown above to obtain the Jacobian matrix of a vector-valued function. The functions in the vector must be defined as Python functions as discussed above. A basic example is shown below for the vector valued function $f = \begin{bmatrix}
xy \\
y \\
ln(x^y)\\
\end{bmatrix}$ at the points where $x = 1, y = 2$. 

In [22]:
def f1(x, y):
    return x * y

def f2(y):
    return y

def f3(x, y):
    return rAd_Var.log(x ** y)

rAd_Var.get_jacobian([f1, f2, f3], ["x","y"], [1, 2])

array([[2., 1.],
       [0., 1.],
       [2., 0.]])

In the event that more variables are defined in the constructor but not used in the functions, their partial derivative is defined in the Jacobian matrix as 0, as shown below with the inclusion of the variable `z`. This gives the user flexibility in adjusting the vector functions to include/exclude a variable where needed. Note: In the arguments passed into the Python functions, the user should only include variables used in the function as arguments (e.g. for `f2`, including only `y` and not both `x` and `y`).

In [23]:
def f1(x, y):
    return x * y

def f2(y):
    return y

def f3(x, y):
    return rAd_Var.log(x ** y)

rAd_Var.get_jacobian([f1, f2, f3], ["x","y","z"], [1, 2, 3])

array([[2., 1., 0.],
       [0., 1., 0.],
       [2., 0., 0.]])

The `get_values` method is then used to return an array of computed values for the functions passed into the method.

In [24]:
rAd_Var.get_values([f1, f2, f3], ["x","y","z"], [1, 2, 3])

array([2., 2., 0.])

### Comparison of `rAd_Var` vs. `Ad_var` in obtaining the Jacobian matrix

While the implementation of the Jacobian matrix for the reverse mode is different, it will return a similar result as the forward mode (implemented in the `Ad_Var` class). We recommend that the user uses the `rAd_Var` class, in the event that the number of inputs is significantly greater than the number of functions, where it will perform more efficiently.

Below we compare the output from the `get_jacobian` method from both methods, based on the more complicated equation $f = \begin{bmatrix}
sin^2(2x) + z^y \\
e^x + z
\end{bmatrix}$ for $x = 1, y = 2, z = 3$, used in the earlier demo.

In [25]:
x = Ad_Var(1, np.array([1, 0, 0]))
y = Ad_Var(2, np.array([0, 1, 0]))
z = Ad_Var(3, np.array([0, 0, 1]))

f = np.array([(Ad_Var.sin(2*x))**2 + z**y, Ad_Var.exp(x) + z])
Ad_Var.get_jacobian(f, 2, 3)

array([[-1.51360499,  9.8875106 ,  6.        ],
       [ 2.71828183,  0.        ,  1.        ]])

In [26]:
def f1(x, z, y):
    return rAd_Var.sin(2*x)**2 + z**y

def f2(x, z):
    return rAd_Var.exp(x) + z

rAd_Var.get_jacobian([f1, f2], ["x","y","z"], [1, 2, 3])

array([[-1.51360499,  9.8875106 ,  6.        ],
       [ 2.71828183,  0.        ,  1.        ]])

## Software Organization

### Directory structure
               
Our intended directory structure is as follows:
```
cs207-FinalProject/
                   ADKit/
                        test/
                            test_autodiff.py
                            test_autodiff_reverse.py
                        AutoDiff.py
                   demo/
                        demo.py               
                   docs/
                        documentation.ipynb
                   LICENSE
                   README.md
                   requirements.txt
                   setup.cfg
                   setup.py
```                 

### Modules

The primary module is a single `AutoDiff.py` file. Contained within it are two classes - the `Ad_Var` class and `rAd_Var` class. 

Instances of these two classes, through interaction with other objects of the same class, are able to compute the value of a function as well as the value of that function's derivative with respect to any input variable. This module is powerful enough to handle both forward and reverse mode of Automatic Differentiation of any function comprised of the following elementary functions:

* Fundamental arithmetic operators (addition, subtraction, multiplication, and division)
* Logarithm (of any base)
* Negation
* Exponentiation ($e^x$ for an `Ad_Var` instance $x$)
* Power and root functions ($x^n$ for some real $n$)
* Trigonometric functions ($\sin(x)$, $\cos(x)$, $\tan(x)$)
* Inverse trigonometric functions ($\arcsin(x)$, $\arccos(x)$, $\arctan(x)$)

Each instance of the `Ad_Var` and `rAd_Var` class in the `AutoDiff` module in the `ADKit` package represents the definition of a set of variables at a particular evaluation point. Through manipulations of these instances (either through fundamental arithmetic operations or built-in methods representing additional elementary functions described earlier), a user has the capability of representing any continuous differentiable function, be it scalar or vector. This was shown earlier via the code demo.

The other modules in the package are stored in the `test` folder and make up the test-suite for `AutoDiff.py`, with more details in the *Testing and Coverage* section below.

### Testing and Coverage
In the `test` folder in the `ADKit` package, there are two separate Python modules `test_autodiff.py` and `test_autodiff_reverse.py`, which together consist of the test-suite for the `AutoDiff` module.

`test_autodiff.py` will contain tests for the methods in the `Ad_Var` class and `test_autodiff_reverse.py` for the `rAd_Var` class, to ensure that the elementary functions return the desired output. Tests are run using pytest. The tests are linked to Travis CI and CodeCov, which will manage continuous integration and code coverage respectively.

###  Installation and Distribution of package 
The package is distributed via PyPI. There is no additional packaging framework included; we believe the scope of this project can be contained within a relatively simple directory structure with few functional python files and should not require additional overhead for users to install and use.

The user is able to install the package in the standard way using `pip`, as shown above in the *How to Use* section. We have also provided a `requirements.txt` file which can be used by the user to create the appropriate environment and ensure that dependencies (i.e. `numpy`) have been installed. Then, to use the package, the user will only need to import `ADKit`.

## Implementation Details

### Class Implementation and Core Attributes

* There are two classes used for forward mode, as well as the extension (reverse mode): `Ad_Var` class and `rAd_Var` class respectively.

* The choice of keeping them as two separate classes is based on the fact that there is limited resuability of code between both implementations - the forward mode determines the derivatives of the variables using the chain rule whereas reverse mode traverses the computational graph in the forward pass and stores both parent-child relationships and the respective partial derivatives of the variables without doing the chain rule.

#### `Ad_Var` Class (Reverse Mode)

* The `Ad_Var`  class will represent the variables that are used in the forward mode of Automatic Differentiation process. In the case of a single input, the instance should be initialized with, `val`, a scalar value of that variable to be evaluated on when calculating both the function and derivative values (as shown in the demo above)

* In the case of multiple inputs, each input is initialized as an `Ad_Var` object, with inputs `val`, a scalar value of that variable and `ders`, a `numpy` array representing the derivative of the input with regards to the other variables. An example is shown below:

In [27]:
x1 = Ad_Var(1, np.array([1, 0, 0]))
x2 = Ad_Var(2, np.array([0, 1, 0]))
x3 = Ad_Var(3, np.array([0, 0, 1]))

* Dunder methods such as "add" and "mul", and other elementary functions are implemented under this class. More information on this is covered below in the *Class Methods* section. 

* As part of the class methods, we have included two static methods, `get_jacobian` and `get_values`, which respectively compute the Jacobian matrix and an array of function values for an array of `Ad_Var` objects. Also, a static method `grid_eval` is included which evaluates the function and its derivative/gradient/jacobian on a grid of points.

* In our implementation, we will also use the try-except method to catch unexpected input types: for example, if the user initializes the variable value of the `Ad_Var` instance with a value of type string, which is not a valid input type.


#### `Ad_Var`: Core Attributes

* `_val`: float value, indicating the function value of the Ad_Var object evaluated at the given point
* `_ders` (for single input): float value, indicating the derivative value of Ad_Var object evaluated at the given point
* `_ders` (for multiple inputs): 1-D array of floats, representing the value of the derivatives of the multiple inputs evaluated at the given point

`_val` and `_ders` attributes are made pseudoprivate to prevent users from manually setting function and derivative values outside of instance initialization

#### `rAd_Var` Class (Reverse Mode)

* The `rAd_Var`  class will represent the variables that are used in the reverse mode of Automatic Differentiation process. In the case of a single input, the instance should be initialized with, `val`, a scalar value of that variable to be evaluated on when calculating both the function and derivative values (as shown in the demo above)

* The initialization of a `rAd_Var` instance does not allow for the input of a derivative value. The implementation necessitates that the derivative of the instance is initialized as None. There is the option for the user to manually set the derivative using the `set_ders` function, if they so wish to.

* The derivatives of the `rAd_Var` object obtained using the `get_ders` method are returned as a numpy array of partial derivative(s) of the input variables with respective to the final function

* As part of the class methods, we have included two static methods, `get_jacobian` and `get_values`, which respectively compute the Jacobian matrix and an array of function values for an array of `rAd_Var` objects. 

* To obtain a Jacobian matrix for vector functions, the user will need to define the functions first before passing them (as Python functions) as arguments for the `get_jacobian` method, together with the variable names and the given values for the inputs, as shown in the demo above.

* This difference in the implementation of the Jacobian matrix is because `rAd_Var` objects can only be defined in the context of one function. Hence, feeding all the functions into the `get_jacobian` method allows for the Jacobian to obtain the respective partial derivatives for each function separately, before combining them and returning them in a single Jacobian matrix.

* In the case of multiple inputs, each input is initialized as an `rAd_Var` object, with inputs `val`.

#### `rAd_Var`: Core Attributes

* `_val`: float value, indicating the function value of the rAd_Var object evaluated at the given point
* `_ders`: instantiated as None and updated via the `get_ders` method which sets the derivative of the final function (with respect to itself) as 1 before using the `get_gradient` helper method to recursively go through all children of the input variables and updating `_ders` with their partial derivative value   
* `parents`: lists containing the parent node(s) of the rAd_Var object; initialized as an empty list and populated at every computation (via class methods)
* `children`: lists of tuples containing the children node(s) of the rAd_Var object; initialized as an empty and populated at every computation (via class methods)
* `visited`: boolean value used to track if a node has been traversed in the reverse pass

### Core Data Structures

In both classes, the following core data structures were used:

* **`numpy` arrays**: 1-D `numpy` arrays are used to keep the gradient vectors as the entire trace is evaluated. `numpy`
provides vectorized operations which will make the overloading of elementary functions much more efficient for
multivariate functions. If a vector function is provided, 2-D `numpy` arrays are used to hold the Jacobian matrix.

* **Dictionaries**: In the `Ad_Var` class, dictionaries are used to keep the results of `grid_eval` function call. Particularly, the keys of the dictionary are points on the grid defined by the user and the corresponding values are the function value and its derivative/gradient/jacobian at the corresponding point. In the `rAd_Var` class, dictionaries are used extensively in the `get_jacobian` method to keep and track the variable inputs and the partial derivatives for each variable separately.

Specifically, in the `rAd_Var` class, where a node is defined as the point at which the computation of a new `rAd_Var` instance is performed, lists are used to store the the parent-child relationships of each node.

* **Lists**: At the every computation of a new `rAd_Var` instance, the new object and the partial derivative of the new function with respect to the input variable are stored as a tuple. This simulates the forward pass of the reverse mode. The tuples in the list are then accessed in the reverse pass of the reverse mode using the `get_ders` method.

### External Dependencies
* `numpy` for implementation of the elementary functions (e.g. sin, sqrt, log and exp), by overloading `numpy` implementations for these functions
* `pytest` and `doctest` for testing
* TravisCI and CodeCov used to manage continuous integration and code coverage

### Elementary Functions and Class Methods

While both the forward mode and reverse mode support a same set of basic operations, comparison operators and elementary functions, the computation of the attributes of the object returned by the functions is vastly different.

At each computation, the reverse mode stores both parent-child relationships and the respective partial derivatives of the variables to each other without doing the chain rule via the attributes `parents` and `children`, covered above. Below are the functions supported by both classes - for more details on how the partial derivatives of the variables are calculated for `rAd_Var`, refer to the code.

#### Elementary Functions / Operators supported by both classes
* `__add__(self, other)` and `__radd__(self, other)`:
    * Other can be a float, int or an `AutoDiff` object
    * Returns an `Ad_Var` or `rAd_Var` object when calculating self + other or other + self


* `__sub__(self, other)` and `__rsub__(self, other)`:
    * Other can be a float, int or an `AutoDiff` object
    * Returns an `Ad_Var` or `rAd_Var` object when calculating self - other or other - self


* `__mul__(self, other)` and `__rmul__(self, other)`:
    * Other can be a float, int or an `AutoDiff` object
    * Returns an `Ad_Var` or `rAd_Var` object when calculating self * other or other * self
    

* `__truediv__(self, other)` and `__rtruediv__(self, other)`:
    * Other can be a float, int or an `AutoDiff` object
    * Returns an `Ad_Var` or `rAd_Var` object when calculating self / other or other / self


* `__pow__(self, other)` and `__rpow__(self, other)`:
    * `other` can be a float, int or an `AutoDiff` object
    * `__rpow__` will require `other` to be a numeric type, otherwise, it will raise a TypeError
    * Returns an `Ad_Var` or `rAd_Var` object when calculating self ** other


* `__neg__(self)`:
    * Returns an `Ad_Var` or `rAd_Var` object when calculating - self


* `__eq__(self, other)`:
    * Returns True if `self._val` == `other._val` and `self._ders` == `other._ders`, returns False otherwise
    
    
* `__ne__(self, other)`:
    * Returns True if `self._val` != `other._val` or `self._ders` != `other._ders`, returns False otherwise
    
    
* `__repr__(self)`:
    * Returns a string representing the value of `self._val` (Value) and the value of `self._ders` (Gradient)
    

*  `sqrt(self)`:
    * Returns an `Ad_Var` or `rAd_Var` object by calling the __pow__ method using self**0.5
    

* `exp(self)`:
    * `Ad_Var`: Returns an `Ad_Var` object with `self._val = np.exp(self._val)` and `self._ders = np.exp(self._val) * self._ders`
    * `rAd_Var`: Returns an `rAd_Var` object with `self._val = np.exp(self._val)` and the `parent` and `children` attributes for both `self` and the new `rAd_Var` object updated accordingly


* `log(self, logbase=np.e)`:
    * Optional argument for `logbase` (can be a float or int). By default, `logbase` is set to the exponential.
    * `Ad_Var`: Returns an `Ad_Var` object with `self._val = np.log(self._val) / np.log(logbase)` and `self._ders = self._ders / (self._val * np.log(logbase))`.
    * `rAd_Var`: Returns an `rAd_Var` object with `self._val = np.log(self._val) / np.log(logbase)` and the `parent` and `children` attributes for both `self` and the new `rAd_Var` object updated accordingly
    
    
* `sin(self)` and `cos(self)` and `tan(self)`:
    * Returns an `Ad_Var` or `rAd_Var` object with the respective class attributes updated accordingly based on the given trigonometric function
    
    
* `arcsin(self)` and `arccos(self)` and `arctan(self)`:
    * Returns an `Ad_Var` or `rAd_Var` object with respective class attributes updated accordingly based on the given inverse trigonometric function  


* `sinh(self)` and `cosh(self)` and `tanh(self)`:
    * Returns an `Ad_Var` or `rAd_Var` object with respective class attributes updated accordingly based on the given hyperbolic function


* `logistic(self)`: 
    * Returns an `Ad_Var` or `rAd_Var` object with respective class attributes updated accordingly based on the logistic (sigmoid) function
    

* `set_val(self, value)`:
    * Set the value of the private attribute `self._val` with `value`


* `set_ders(self, derivatives)`:
    * Set the value of the private attribute `self._ders` with `derivatives`
    
    
* `get_val(self)`:
    * Returns the value of the attribute `self._val` 
    
    
#### Specific `Ad_Var` class methods
* `__init__(self, val, ders=1)`:
    * Sets `self._val` to the argument `val`
    * Sets `self._ders` to the argument `ders`
  
  
* `get_ders(self)`:
    * Returns the value of the attribute `self._ders` 
    
    
* `get_jacobian(functions_array, functions_dim, vars_dim)`:
    * Static method that returns the Jacobian matrix for a given array of `Ad_Var` objects 
    
    
* `get_values(functions_array)`:
    * Static method that returns an array of function values for a given array of `Ad_Var` objects
   
   
* `grid_eval(func_string, vars_strings, Ad_Vars_list, grid)`:
    * Static method that evaluates a function and its derivative/gradient/jacobian on a grid of points. A dictionary is returned where each key is a point of the grid and the value is a tuple with the first element being the value of the function at this point and second element is the derivative/gradient/jacobian evaluated at this point.


#### Specific `rAd_Var` class methods
* `__init__(self, val, ders=1)`:
    * Sets `self._val` to the argument `val`
    * Initializes `self._ders` as `None`
    * Initializes `self.parents` as an empty list
    * Initializes `self.children` as an empty list
    * Initializes `self.visited` as `False`
    

* `get_ders(self)`:
    * Sets the derivative of the final function (with respect to itself) as 1 before using the `get_gradient` helper method, which recursively traverses all children of the input variables and updating `_ders` with their partial derivative value   
    * Returns `gradient_matrix`, a `numpy` array consisting of the partial derivatives of input variables used to compute the final function
    
    
* `get_jacobian(functions_array, var_list, var_values)`:
    * Static method that returns the Jacobian matrix for a vector of Python functions, with given variable names and values for the variables used as arguments in these functions
    * Instantiation of `rAd_Var` objects is done within this method based on the functions and variables passed in
    
    
* `get_values(functions_array, var_list, var_values)`:
    * Static method that returns an array of function values for a vector of Python functions, with given variable names and values for the variables used as arguments in these functions
    * Instantiation of `rAd_Var` objects is done within this method based on the functions and variables passed in

## Extension (Reverse Mode)

### Description 

As part of an extension from the earlier milestone, we have implemented the reverse mode of Automatic Differentiation using the `rAd_Var` class, with details on the class (data structures, attributes and methods) elaborated and demonstrated above.

In terms of efficiency, the forward mode is more efficient when the number of functions to evaluate is much greater than the number of inputs, whereas the reverse mode, which computes the Jacobian-transpose-vector-product is more efficient when the number of inputs is much greater than the number of functions.

Having the `rAd_Var` class allows the user of this package the flexibility to choose between the two modes of automatic differentiation, depending on the vector functions and variables that they will carry out Automatic Differentiation on.

### Additional information / background

Unlike the forward mode of Automatic Differentiation, the reverse mode of Automatic Differentiation consists of two stages - 1) forward pass, followed by the 2) reverse pass (also popularly known as backward propagation).

With a function $f$ which takes in $n$ inputs, $x_1, x_2, ..., x_n$ and produces a single output, the reverse mode will return derivatives of the function $\dfrac{\partial f}{\partial x_{i}}$ for all $i$.

The following references and builds on [CS207 2019 Lecture 12 materials](https://harvard-iacs.github.io/2019-CS207/lectures/lecture12/notebook/) (Sondak):

**Forward pass**

In the forward pass, the evaluation of the function is carried out at each elementary function, $f_i$, of the entire function, $f$. What this means is that for $i = n + 1, n + 2, ..., N$, where $\pi(i)$ denotes the set of "parents" of $x_i$.

$$x_i = f_i(x_\pi(i))$$ 

E.g. for $x_{3} = x_{1}x_{2}$, where the elementary function, $f_3$, is a multiplication of two input variables, thus forming a node in the computational graph. Here, $\pi(3) = (x_1, x_2)$. This is important in the forward pass, which stores the partial derivatives of every node with respect to its "parents", as denoted by $\pi(i)$. In the example above, where $x_{3} = x_{1}x_{2}$, we store $\dfrac{\partial x_{3}}{\partial x_{1}}$ and $\dfrac{\partial x_{3}}{\partial x_{2}}$.

This can be best illustrated using an example where $$ f (x, y) = 2xy - \exp(xy)$$ at the point $x = 1, y = 2$

Here, to generate the forward trace, we calculate the partial derivatives of a node with regards to its children:

| Node   | Current Value      | Numerical Value           | $\partial_{1}$       | $\partial_{1}$ Value  | $\partial_{2}$ | $\partial_{2}$ Value |
| :---: | :----------------------------: | :-----------: | :----------------------------: | :-----------------:  | :-----------------: | :-----------------: |
| $x_{1}$ | $x$                      | $1$         | $1$                        | $1$     | $0$     | $0$   |
| $x_{2}$ | $y$                      | $2$         | $0$                        | $0$     | $1$     | $1$   | 
| $x_{3}$ | $x_{1}x_{2}$             | $2$         | $x_{2}$                    | $2$     | $x_{1}$ | $1$   |
| $x_{4}$ | $2x_{3}$                 | $4$         | $2$                        | $2$     | $-$     | $-$   |
| $x_{5}$ | $\exp\left(x_{3}\right)$ | $e^{2}$     | $\exp\left(x_{3}\right)$   | $e^{2}$ | $-$     | $-$   |
| $x_{6}$ | $-x_5$                   | $-e^{2}$    | $-1$                       | $-1$    | $-$     | $-$   |
| $x_{7}$ | $x_{4} + x_{6}$          | $4 - e^{2}$ | $1$                        | $1$     | $1$     | $1$   |

**Reverse pass**

Following the forward pass, the reverse pass starts from the final function to be differentiated, $f_N$, setting $\overline{x}_{N} = \dfrac{\partial f}{\partial x_{N}} = 1$ (since $f = x_{N}$).

Then, using the chain rule, it traverses the computational graph to obtain values for the partial derivative of every variable in the computational graph, $x_i$: 

\begin{align}
  \overline{x}_{i} = \dfrac{\partial f}{\partial x_{i}} = \sum_{\text{j a child of i}}{\dfrac{\partial f}{\partial x_{j}}\dfrac{\partial x_{j}}{\partial x_{i}}}.
\end{align}

This is done recursively until the partial derivatives of the function with respect to the $n$ inputs, $x_1, x_2, ..., x_n$ are computed.

Using the same example above, we recursively go through every variable($x_i$) in the computational trace shown above. The computation of the gradient of each variable accesses the partial derivative with respect to its children that has been previously computed and stored during the forward pass.

$$\overline{x}_{7} = \dfrac{\partial f}{\partial x_{7}} = 1$$

$$\overline{x}_{6} = \dfrac{\partial f}{\partial x_{7}}\dfrac{\partial x_{6}}{\partial x_{7}} = 1 \cdot 1 = 1$$

$$\overline{x}_{5} = \dfrac{\partial f}{\partial x_{6}}\dfrac{\partial x_{6}}{\partial x_{5}} = 1 \cdot (-1) = -1$$

$$\overline{x}_{4} = \dfrac{\partial f}{\partial x_{7}}\dfrac{\partial x_{7}}{\partial x_{4}} = 1 \cdot 1 = 1$$

$$\overline{x}_{3} = \dfrac{\partial f}{\partial x_{5}}\dfrac{\partial x_{5}}{\partial x_{3}} + \dfrac{\partial f}{\partial x_{4}}\dfrac{\partial x_{4}}{\partial x_{3}}= (-1 \cdot e^{2}) + (1\cdot 2) = 2 - e^{2}$$

$$\overline{x}_{2} = \dfrac{\partial f}{\partial x_{3}}\dfrac{\partial x_{3}}{\partial x_{2}} = (2 - e^{2}) \cdot 1 = 2 - e^{2}$$

$$\overline{x}_{1} = \dfrac{\partial f}{\partial x_{3}}\dfrac{\partial x_{3}}{\partial x_{1}} = (2 - e^{2}) \cdot 2 = 4 - 2e^{2}$$

This gives us the gradient of:

\begin{align}
  \nabla f &= 
  \begin{bmatrix} 
    4 - 2e^{2} \\ 
    2 - e^{2} 
  \end{bmatrix}
\end{align}

which is identical to what we would have calculated using symbolic differentiation for the function $f (x, y) = 2xy - \exp(xy)$:

$$\nabla f = \begin{bmatrix} 2y - \exp\left(xy\right)y \\ 2x - \exp\left(xy\right)x \end{bmatrix}$$

Reverse mode of Automatic Differentiation is supported by the `rAd_Var` class in the `AutoDiff` module. Further details on using the class is covered in the *How to use `ADKit` (Reverse Mode)* section of the Documentation

#### References

* [A Hitchhiker’s Guide to Automatic Differentiation](https://link.springer.com/article/10.1007/s11075-015-0067-6)
* [A simple explanation of reverse-mode automatic differentiation](https://justindomke.wordpress.com/2009/03/24/a-simple-explanation-of-reverse-mode-automatic-differentiation/)
* Harvard CS207 2019 course materials